<a href="https://colab.research.google.com/github/nasifnur307-blip/biomarkerlink/blob/main/notebooks/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy scipy matplotlib scikit-learn pandas networkx pgmpy causal-learn


In [2]:
!pip install causal-learn==0.1.4.3



In [3]:
# lowercase causallearn
from causallearn.search.ConstraintBased.PC import pc


In [4]:
from sklearn.preprocessing import StandardScaler


In [5]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from causallearn.search.ConstraintBased.PC import pc

print("Ready.")


Ready.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/nasifnur307-blip/biomarkerlink.git


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'biomarkerlink' already exists and is not an empty directory.


In [7]:
import os

folders = [
    "data/raw",
    "data/processed",
    "results/graphs",
    "results/models",
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Folders created.")


Folders created.


In [8]:
df = pd.read_csv('/content/drive/MyDrive/NeuroFedMeta/data/CHBMIT/chbmit_preprocessed_data.csv')

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Label distribution:\n", df['Outcome'].value_counts())

Dataset shape: (2097150, 24)
Columns: ['# FP1-F7', 'C3-P3', 'C4-P4', 'CZ-PZ', 'F3-C3', 'F4-C4', 'F7-T7', 'F8-T8', 'FP1-F3', 'FP2-F4', 'FP2-F8', 'FT10-T8', 'FT9-FT10', 'FZ-CZ', 'P3-O1', 'P4-O2', 'P7-O1', 'P7-T7', 'P8-O2', 'T7-FT9', 'T7-P7', 'T8-P8-0', 'T8-P8-1', 'Outcome']
Label distribution:
 Outcome
0.0    1048575
1.0    1048575
Name: count, dtype: int64


In [9]:
# Features = EEG channels (first 23 columns)
X = df.iloc[:, :-1].values
# Outcome column = inter-/pre-/ictal labels
y = df['Outcome'].values

print("Features shape:", X.shape)
print("Labels shape:", y.shape)


Features shape: (2097150, 23)
Labels shape: (2097150,)


In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [11]:
# Inter-ictal: Outcome = 0
X_inter = X_scaled[y == 0]

# Pre-ictal: Outcome = 1 (adjust if you have explicit pre-ictal vs ictal labels)
X_pre = X_scaled[y == 1]

print("Inter-ictal shape:", X_inter.shape)
print("Pre-ictal shape:", X_pre.shape)


Inter-ictal shape: (1048575, 23)
Pre-ictal shape: (1048575, 23)


In [12]:

alpha = 0.05


is_identical = df[df.columns[-3]].equals(df[df.columns[-2]])
print(f"Are columns '{df.columns[-3]}' and '{df.columns[-2]}' identical? {is_identical}")

if is_identical:
    print(f"Dropping column '{df.columns[-2]}' due to duplication.")
    # Re-extract features, excluding the duplicate column
    features_to_use = [col for col in df.columns[:-1] if col != df.columns[-2]]
    X_fixed = df[features_to_use].values


    scaler_fixed = StandardScaler()
    X_scaled_fixed = scaler_fixed.fit_transform(X_fixed)


    X_inter_fixed = X_scaled_fixed[y == 0]
    X_pre_fixed = X_scaled_fixed[y == 1]

    print("Running PC algorithm with fixed data...")
    causal_graph_inter = pc(data=X_inter_fixed, alpha=alpha, method_ind='hsic')
else:
    print("Columns are not identical, running PC with original data...")
    causal_graph_inter = pc(data=X_inter, alpha=alpha, method_ind='hsic')


print("PC algorithm finished for inter-ictal state.")

Are columns 'T8-P8-0' and 'T8-P8-1' identical? True
Dropping column 'T8-P8-1' due to duplication.
Running PC algorithm with fixed data...


  0%|          | 0/22 [00:00<?, ?it/s]

PC algorithm finished for inter-ictal state.


In [13]:
alpha = 0.1 # Increased alpha value
causal_graph_inter = pc(data=X_inter_fixed, alpha=alpha, method_ind='hsic')
print("PC algorithm finished for inter-ictal state.")

  0%|          | 0/22 [00:00<?, ?it/s]

PC algorithm finished for inter-ictal state.


In [14]:
import networkx as nx
import matplotlib.pyplot as plt


G = nx.DiGraph()




In [15]:
# Get the NetworkX graph directly from the CausalGraph object
G = causal_graph_inter.to_nx_graph()

# Get the feature names from the original dataframe, excluding the 'Outcome' and the dropped duplicate column
# This assumes 'df' is still available and the duplicate column handling was consistent
if 'T8-P8-1' in df.columns:
    feature_names = [col for col in df.columns[:-1] if col != 'T8-P8-1']
else:
    feature_names = df.columns[:-1].tolist()

# Check if a graph was actually generated
if G is None:
    print("No causal graph could be generated or the graph is empty.")
else:
    # Create a mapping for relabeling nodes if feature names are different from original indices
    # Ensure the number of feature names matches the number of nodes in the graph
    if len(feature_names) == G.number_of_nodes():
        mapping = {i: name for i, name in enumerate(feature_names)}
        G = nx.relabel_nodes(G, mapping)
    else:
        print("Warning: Number of feature names does not match number of nodes in the graph. Skipping relabeling.")

    # Plot the graph
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G) # You can try other layouts like circular_layout, shell_layout
    nx.draw_networkx_nodes(G, pos, node_size=700, node_color='lightblue')
    nx.draw_networkx_edges(G, pos, arrowstyle='->', arrowsize=20, edge_color='gray')
    nx.draw_networkx_labels(G, pos, font_size=8)
    plt.title('Causal Graph for Inter-ictal State')
    plt.axis('off') # Hide axes
    plt.show()

No causal graph could be generated or the graph is empty.


In [16]:
# Get the NetworkX skeleton (undirected graph) from the CausalGraph object
G_skel = causal_graph_inter.to_nx_skeleton()

# Get the feature names from the original dataframe, excluding the 'Outcome' and the dropped duplicate column
# This assumes 'df' is still available and the duplicate column handling was consistent
if 'T8-P8-1' in df.columns:
    feature_names = [col for col in df.columns[:-1] if col != 'T8-P8-1']
else:
    feature_names = df.columns[:-1].tolist()

# Check if a skeleton was actually generated
if G_skel is None:
    print("No causal graph skeleton could be generated or the skeleton is empty.")
else:
    # Create a mapping for relabeling nodes if feature names are different from original indices
    if len(feature_names) == G_skel.number_of_nodes():
        mapping = {i: name for i, name in enumerate(feature_names)}
        G_skel = nx.relabel_nodes(G_skel, mapping)
    else:
        print("Warning: Number of feature names does not match number of nodes in the skeleton. Skipping relabeling.")

    # Plot the skeleton
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G_skel) # You can try other layouts like circular_layout, shell_layout
    nx.draw_networkx_nodes(G_skel, pos, node_size=700, node_color='lightcoral')
    nx.draw_networkx_edges(G_skel, pos, edge_color='gray') # No arrows for undirected graph
    nx.draw_networkx_labels(G_skel, pos, font_size=8)
    plt.title('Causal Graph Skeleton for Inter-ictal State (Undirected Associations)')
    plt.axis('off') # Hide axes
    plt.show()

No causal graph skeleton could be generated or the skeleton is empty.


In [17]:
print(type(causal_graph_inter))
print(dir(causal_graph_inter))

<class 'causallearn.graph.GraphClass.CausalGraph'>
['G', 'PC_elapsed', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', 'ci_test', 'definite_UC', 'definite_non_UC', 'draw_nx_graph', 'draw_pydot_graph', 'find_adj', 'find_arrow_heads', 'find_bi_directed', 'find_cond_sets', 'find_cond_sets_with_mid', 'find_cond_sets_without_mid', 'find_fully_directed', 'find_kites', 'find_tails', 'find_triangles', 'find_undirected', 'find_unshielded_triples', 'is_fully_directed', 'is_undirected', 'labels', 'max_degree', 'neighbors', 'nx_graph', 'nx_skel', 'prt_m', 'rearrange', 'redundant_nodes', 'sepset', 'set_ind_test', 'test', 'to_nx_graph', 'to_nx_skeleton']


# Task
Investigate why the PC algorithm is not generating a causal graph or skeleton for the inter-ictal state. This includes checking the internal state of the `causal_graph_inter` object (e.g., number of nodes and edges) to understand if the PC algorithm is genuinely finding no causal links with the current preprocessing and alpha level, or if there's an issue with the graph conversion.

## Evaluate Existing Preprocessing

### Subtask:
Analyze how the existing preprocessing of the CHBMIT dataset might be affecting the detectability of causal links. This includes considering if important temporal or spectral information was lost, or if normalization/feature extraction methods have obscured dependencies suitable for the PC algorithm. We will discuss the trade-offs between using a preprocessed dataset and raw data for causal inference.


## Analyze Existing Preprocessing Impact on Causal Links

This section reviews the current preprocessing applied to the CHBMIT dataset and analyzes its potential implications for the detectability of causal links using the PC algorithm. We will consider how `StandardScaler` and the removal of duplicate columns might affect the underlying data structure relevant for causal inference, and discuss the trade-offs involved.

### Review of Preprocessing Steps

In the preceding steps, the following preprocessing actions were performed on the CHBMIT dataset:

1.  **Data Loading**: The dataset `chbmit_preprocessed_data.csv` was loaded into a Pandas DataFrame `df`.
2.  **Feature and Label Separation**: The last column, 'Outcome', was separated as the label `y`, and the remaining columns were designated as features `X`.
3.  **Standardization**: The features `X` were standardized using `sklearn.preprocessing.StandardScaler` to `X_scaled`. This transforms the data to have zero mean and unit variance.
4.  **Duplicate Column Handling**: A check for identical columns `T8-P8-0` and `T8-P8-1` was performed. Since they were found to be identical, `T8-P8-1` was dropped, and the features were re-extracted and scaled as `X_fixed` and `X_scaled_fixed`.
5.  **State Separation**: The data was split into inter-ictal (`y == 0`) and pre-ictal (`y == 1`) states, resulting in `X_inter_fixed` and `X_pre_fixed` (after duplicate column handling).
6.  **PC Algorithm Application**: The PC algorithm was applied to `X_inter_fixed` to infer a causal graph.

Now we will analyze the implications of these steps for causal inference.

### Impact of Standardization (`StandardScaler`)

**StandardScaler** transforms features to have a mean of 0 and a standard deviation of 1. While beneficial for many machine learning algorithms that assume normally distributed or scaled data, its impact on causal inference can be nuanced:

*   **Preservation of Linear Relationships**: Standardization typically preserves linear relationships between variables. If the underlying causal links are linear, `StandardScaler` should not fundamentally alter the conditional independencies necessary for the PC algorithm.
*   **Impact on Non-linear Relationships**: For non-linear relationships or tests that are sensitive to the absolute scale of the data, standardization might obscure or amplify certain patterns. However, the `hsic` (Hilbert-Schmidt Independence Criterion) used in the PC algorithm is designed to detect general (including non-linear) dependencies and is generally less sensitive to scaling changes than parametric tests.
*   **Variance and Distribution**: By enforcing unit variance, `StandardScaler` removes information about the original scale and spread of each feature. This might be a concern if the absolute magnitude of variability in an EEG channel carries causal information, which is then lost. However, for identifying *conditional independencies*, the relative information content is often more important than absolute scales.
*   **Outlier Influence**: Standardization can be sensitive to outliers. While it doesn't remove them, extreme values can disproportionately influence the mean and standard deviation, potentially compressing the range of other data points. If outliers are causally relevant, this transformation could distort their perceived influence.

In the context of the PC algorithm, which relies on conditional independence tests, `StandardScaler` is generally considered acceptable as long as the independence test itself is robust to scaling (like `hsic`). It ensures that all features contribute equally to distance calculations or similarity measures within the independence test, preventing features with larger scales from dominating.

### Impact of Duplicate Column Handling (Removing 'T8-P8-1')

**Duplicate column handling** involved identifying and removing 'T8-P8-1' because it was identical to 'T8-P8-0'.

*   **Benefits**: Removing truly identical columns is generally a good practice in data analysis and causal inference. Duplicate features provide no new information and can unnecessarily increase computational complexity or introduce redundancy in models. For constraint-based algorithms like PC, having identical features would mean they are perfectly correlated, which could lead to trivial dependencies or issues in independence testing if not handled properly.
*   **No Information Loss**: Since the columns were identical, removing one of them does not result in the loss of unique information. The causal relationships that might exist with 'T8-P8-0' are fully preserved, as 'T8-P8-1' was merely a redundant representation of the same signal.
*   **Simplification**: It simplifies the causal discovery process by reducing the number of variables the PC algorithm needs to consider, potentially making the resulting graph cleaner and easier to interpret without sacrificing any genuine causal signal.

In summary, the removal of 'T8-P8-1' is a benign preprocessing step for causal inference, as it eliminates redundancy without losing any distinct information relevant to the causal structure.

### Potential Loss of Information and Impact on PC Algorithm

The current preprocessing steps, while standard, might have implications for specific types of causal links, especially those related to temporal or spectral dynamics:

*   **Loss of Temporal Information**: The current dataset `chbmit_preprocessed_data.csv` is presented as a snapshot of features derived from EEG channels. It's unclear from the provided context whether this data retains its inherent temporal sequence or if it has been aggregated (e.g., averaged over time windows, or treated as independent samples). If temporal ordering and dependencies (e.g., X causes Y at a later time point) are crucial for the causal relationships in EEG, and if the data has lost its sequential nature, then the PC algorithm, which typically infers causal graphs from i.i.d. (independent and identically distributed) samples, might struggle to capture these time-lagged causal effects. The PC algorithm implicitly assumes contemporaneous relationships unless explicitly extended for time-series data.

*   **Loss of Spectral Information**: Similarly, the feature names (e.g., 'FP1-F7', 'C3-P3') suggest that these are raw or band-filtered EEG channel values. If specific frequency band power or cross-frequency coupling carries causal information, and the preprocessing involved simple amplitude values without explicit spectral decomposition (e.g., into alpha, beta, gamma bands), then this information might not be optimally represented for causal discovery. The `hsic` test can capture general dependencies, but if the causal links are intrinsically spectral, representing them only in the time domain (or simple amplitude domain) might make them harder to detect.

*   **Difficulty for PC Algorithm**: The PC algorithm relies on identifying conditional independencies. If the preprocessing has inadvertently transformed causally related variables in a highly non-linear or aggregated manner that obscures these independencies (even if `hsic` is used for non-linear detection), then the PC algorithm might produce a sparse or inaccurate graph. For example, if a causal effect is very weak but consistent over time, and averaging reduces its signal-to-noise ratio, it might fail independence tests.

*   **Assumption of i.i.d. Data**: A fundamental assumption of many causal discovery algorithms, including PC, is that the data points are independent and identically distributed (i.i.d.). While we are analyzing two different states (inter-ictal and pre-ictal), within each state, the individual rows of the dataframe are treated as i.i.d. samples. For highly correlated time-series data like EEG, this assumption is often violated. This violation can lead to spurious causal links or an inability to detect true ones. More sophisticated causal inference methods for time series (e.g., Granger causality, VAR models, or extensions of PC for time series) might be required if the temporal structure is critical.

### Trade-offs: Preprocessed Data vs. Raw Data for Causal Inference

Choosing between a preprocessed dataset and raw EEG data for causal inference involves several trade-offs:

**Benefits of Using Preprocessed Data (as currently done):**

*   **Reduced Noise and Artifacts**: Preprocessing often includes steps like artifact removal (e.g., eye blinks, muscle activity) and noise reduction, leading to cleaner signals. This can improve the signal-to-noise ratio, potentially making true causal effects more detectable and reducing spurious correlations.
*   **Dimensionality Reduction/Feature Engineering**: The preprocessed dataset likely represents features that are already aggregated or extracted (e.g., time-averaged EEG channel values). This can simplify the data and reduce the dimensionality, which is beneficial for algorithms like PC that can become computationally expensive with many variables.
*   **Standardization**: As discussed, `StandardScaler` ensures features are on a comparable scale, which can prevent features with larger absolute values from dominating conditional independence tests, especially in distance-based measures.
*   **Computational Efficiency**: Working with preprocessed, reduced-dimensionality data is significantly faster for causal discovery algorithms compared to raw, high-resolution time series data.

**Drawbacks of Using Preprocessed Data for Causal Inference:**

*   **Loss of Temporal/Spectral Resolution**: The most significant drawback is the potential loss of critical temporal and spectral information. If causal links manifest as time-lagged effects (e.g., one brain region influencing another milliseconds later) or in specific frequency bands, and the preprocessing flattens or aggregates this information, these causal links may become undetectable.
*   **Obscured Non-linearities**: While HSIC is robust, if preprocessing applies strong non-linear transformations or aggregations, it might obscure subtle non-linear causal dependencies that would have been evident in raw data.
*   **Assumption of i.i.d. Samples**: Aggregation of time-series data into 'snapshots' forces the assumption of i.i.d. samples, which might violate the temporal autocorrelation inherent in EEG signals, leading to biased causal inferences.
*   **Dependence on Preprocessing Choices**: The causal graph inferred is highly dependent on the choices made during preprocessing. If these choices are suboptimal or based on assumptions that don't hold for the causal question, the results can be misleading.

**Benefits of Using Raw EEG Data for Causal Inference:**

*   **Preservation of Full Information**: Raw data retains all temporal, spectral, and spatial information, allowing for a more comprehensive exploration of potential causal links, including time-lagged effects and frequency-specific interactions.
*   **Flexibility for Advanced Causal Models**: Raw data is suitable for advanced causal inference methods designed for time series (e.g., Granger Causality, Dynamic Causal Modeling, or extensions of PC for time series) that can explicitly model temporal dependencies.

**Drawbacks of Using Raw EEG Data:**

*   **High Dimensionality and Complexity**: Raw EEG data is extremely high-dimensional and noisy, requiring significant computational resources and sophisticated methods to analyze effectively.
*   **Increased Risk of Spurious Correlations**: Without proper noise reduction and artifact removal, raw data can lead to many spurious correlations, complicating causal discovery.
*   **Difficult to Apply Standard PC Algorithm**: The standard PC algorithm is generally not designed for raw time-series data due to the i.i.d. assumption and high dimensionality, making its direct application problematic.

**Conclusion on Trade-offs:**

For the current subtask using the provided `chbmit_preprocessed_data.csv` and the PC algorithm, the preprocessing steps are largely acceptable given the tools and data format. `StandardScaler` and duplicate removal are standard and generally benign for conditional independence tests with robust statistics like HSIC. However, it is crucial to acknowledge that the *initial creation* of this preprocessed CSV (which is outside the scope of the current notebook) might have involved significant aggregation or feature extraction that could have already sacrificed critical temporal or spectral information relevant to true underlying causal brain mechanisms. If deeper insights into time-dependent or frequency-specific causal dynamics are required, a move towards raw EEG data combined with time-series-aware causal inference methods would be necessary.

### Trade-offs: Preprocessed Data vs. Raw Data for Causal Inference

Choosing between a preprocessed dataset and raw EEG data for causal inference involves several trade-offs:

**Benefits of Using Preprocessed Data (as currently done):**

*   **Reduced Noise and Artifacts**: Preprocessing often includes steps like artifact removal (e.g., eye blinks, muscle activity) and noise reduction, leading to cleaner signals. This can improve the signal-to-noise ratio, potentially making true causal effects more detectable and reducing spurious correlations.
*   **Dimensionality Reduction/Feature Engineering**: The preprocessed dataset likely represents features that are already aggregated or extracted (e.g., time-averaged EEG channel values). This can simplify the data and reduce the dimensionality, which is beneficial for algorithms like PC that can become computationally expensive with many variables.
*   **Standardization**: As discussed, `StandardScaler` ensures features are on a comparable scale, which can prevent features with larger absolute values from dominating conditional independence tests, especially in distance-based measures.
*   **Computational Efficiency**: Working with preprocessed, reduced-dimensionality data is significantly faster for causal discovery algorithms compared to raw, high-resolution time series data.

**Drawbacks of Using Preprocessed Data for Causal Inference:**

*   **Loss of Temporal/Spectral Resolution**: The most significant drawback is the potential loss of critical temporal and spectral information. If causal links manifest as time-lagged effects (e.g., one brain region influencing another milliseconds later) or in specific frequency bands, and the preprocessing flattens or aggregates this information, these causal links may become undetectable.
*   **Obscured Non-linearities**: While HSIC is robust, if preprocessing applies strong non-linear transformations or aggregations, it might obscure subtle non-linear causal dependencies that would have been evident in raw data.
*   **Assumption of i.i.d. Samples**: Aggregation of time-series data into 'snapshots' forces the assumption of i.i.d. samples, which might violate the temporal autocorrelation inherent in EEG signals, leading to biased causal inferences.
*   **Dependence on Preprocessing Choices**: The causal graph inferred is highly dependent on the choices made during preprocessing. If these choices are suboptimal or based on assumptions that don't hold for the causal question, the results can be misleading.

**Benefits of Using Raw EEG Data for Causal Inference:**

*   **Preservation of Full Information**: Raw data retains all temporal, spectral, and spatial information, allowing for a more comprehensive exploration of potential causal links, including time-lagged effects and frequency-specific interactions.
*   **Flexibility for Advanced Causal Models**: Raw data is suitable for advanced causal inference methods designed for time series (e.g., Granger Causality, Dynamic Causal Modeling, or extensions of PC for time series) that can explicitly model temporal dependencies.

**Drawbacks of Using Raw EEG Data:**

*   **High Dimensionality and Complexity**: Raw EEG data is extremely high-dimensional and noisy, requiring significant computational resources and sophisticated methods to analyze effectively.
*   **Increased Risk of Spurious Correlations**: Without proper noise reduction and artifact removal, raw data can lead to many spurious correlations, complicating causal discovery.
*   **Difficult to Apply Standard PC Algorithm**: The standard PC algorithm is generally not designed for raw time-series data due to the i.i.d. assumption and high dimensionality, making its direct application problematic.

**Conclusion on Trade-offs:**

For the current subtask using the provided `chbmit_preprocessed_data.csv` and the PC algorithm, the preprocessing steps are largely acceptable given the tools and data format. `StandardScaler` and duplicate removal are standard and generally benign for conditional independence tests with robust statistics like HSIC. However, it is crucial to acknowledge that the *initial creation* of this preprocessed CSV (which is outside the scope of the current notebook) might have involved significant aggregation or feature extraction that could have already sacrificed critical temporal or spectral information relevant to true underlying causal brain mechanisms. If deeper insights into time-dependent or frequency-specific causal dynamics are required, a move towards raw EEG data combined with time-series-aware causal inference methods would be necessary.


## Investigate causal graph properties

### Subtask:
Investigate why the PC algorithm is not generating a causal graph or skeleton for the inter-ictal state. This includes checking the internal state of the `causal_graph_inter` object (e.g., number of nodes and edges) to understand if the PC algorithm is genuinely finding no causal links with the current preprocessing and alpha level, or if there's an issue with the graph conversion.


**Reasoning**:
To investigate why the PC algorithm is not generating a causal graph or skeleton, I will check the number of nodes and edges in both the NetworkX graph and skeleton representations derived from the `causal_graph_inter` object. This will help determine if the graph is empty or if there's a problem with the conversion.



In [19]:
import networkx as nx

# 1. Access the underlying NetworkX graph representation
inter_graph_nx = causal_graph_inter.to_nx_graph()

# 2. Print the number of nodes and edges in inter_graph_nx
if inter_graph_nx:
    print(f"Inter-ictal Causal Graph (Directed):\n  Number of nodes: {inter_graph_nx.number_of_nodes()}\n  Number of edges: {inter_graph_nx.number_of_edges()}")
else:
    print("Inter-ictal Causal Graph (Directed): No graph generated or graph is empty.")

# 3. Access the underlying NetworkX skeleton (undirected graph) representation
inter_skeleton_nx = causal_graph_inter.to_nx_skeleton()

# 4. Print the number of nodes and edges in inter_skeleton_nx
if inter_skeleton_nx:
    print(f"Inter-ictal Causal Graph Skeleton (Undirected):\n  Number of nodes: {inter_skeleton_nx.number_of_nodes()}\n  Number of edges: {inter_skeleton_nx.number_of_edges()}")
else:
    print("Inter-ictal Causal Graph Skeleton (Undirected): No skeleton generated or skeleton is empty.")

Inter-ictal Causal Graph (Directed): No graph generated or graph is empty.
Inter-ictal Causal Graph Skeleton (Undirected): No skeleton generated or skeleton is empty.


## Consider Inherent Dataset Properties

### Subtask:
Discuss whether the inherent characteristics of the CHBMIT EEG dataset (e.g., high noise, non-stationarity, complex multivariate relationships) themselves make it challenging for constraint-based algorithms like PC to find clear causal structures. We will explore if other causal inference methods might be more suitable if the issue is fundamental to the data type.


## Consider Inherent Dataset Properties

### Impact of CHBMIT EEG Dataset Characteristics on PC Algorithm

The CHBMIT EEG dataset, like most neurophysiological recordings, possesses several inherent characteristics that can significantly challenge constraint-based causal discovery algorithms such as the PC algorithm:

1.  **High Noise Levels**: EEG signals are notoriously noisy. They are susceptible to biological artifacts (e.g., eye blinks, muscle activity, heartbeats) and environmental interference (e.g., power line noise). While preprocessing aims to reduce noise, it's impossible to eliminate it entirely. High noise levels can obscure genuine causal relationships, leading to false negatives (missing true links) or false positives (identifying spurious links) in independence tests, making the PC algorithm less reliable.

2.  **Non-stationarity**: Brain activity, especially during dynamic processes like seizures or changes in cognitive state, is inherently non-stationary. This means that the statistical properties of the EEG signals (mean, variance, spectral content, and crucially, their causal relationships) change over time. The standard PC algorithm assumes stationarity and i.i.d. (independent and identically distributed) observations, which is often violated in EEG data. Applying PC to non-stationary data can yield misleading causal structures that represent an average or transient state, rather than a stable underlying mechanism.

3.  **Complex Multivariate Relationships**: EEG data involves many channels recording activity from different brain regions, reflecting a highly complex and interconnected system. Causal relationships in the brain are often non-linear, time-lagged, and involve feedback loops. The PC algorithm, while capable of detecting non-linear dependencies (when `hsic` is used), is primarily designed for acyclic graphs. Furthermore, its reliance on conditional independence tests can become computationally prohibitive and statistically unstable with a large number of variables, especially when high-order conditional independencies are required.

4.  **Temporal Dependencies**: The PC algorithm, in its standard form, infers contemporaneous causal relationships. However, neural interactions are fundamentally dynamic and time-dependent. One brain region might influence another with a short delay. If the data has been aggregated into static 'snapshots' without preserving temporal order, these crucial time-lagged causal links cannot be discovered by standard PC.

### Suitability of Other Causal Inference Methods for EEG Data

Given the challenges posed by EEG data's inherent characteristics, several other causal inference methods might be more suitable, particularly those designed for time-series data:

1.  **Granger Causality (GC)**: This is a popular method for time-series data, defining causality based on predictability: if past values of variable X help predict future values of variable Y better than past values of Y alone, then X Granger-causes Y. GC is well-suited for detecting time-lagged causal influences, which are prevalent in neural circuits. It can be applied in both time-domain (vector autoregressive models) and frequency-domain (spectral Granger causality) to identify directional flow of information.

2.  **Dynamic Causal Modeling (DCM)**: A model-based approach where researchers specify a plausible network model of interacting brain regions and then use Bayesian inference to estimate the parameters of the model, including the strength and direction of causal connections, by fitting it to observed neurophysiological data. DCM explicitly accounts for non-stationarity (through model parameters) and provides a more biophysically informed interpretation of causal influences. It requires strong prior knowledge of the network structure but can infer hidden states and context-dependent changes.

3.  **Transfer Entropy (TE)**: A non-parametric measure of directed information transfer between two time series. TE quantifies how much better the future state of one time series can be predicted by knowing the past states of another time series, beyond what can be predicted from its own past. It is model-free, can capture non-linear dependencies, and is robust to some forms of noise. TE is particularly useful for analyzing information flow in complex systems like the brain.

4.  **LiNGAM (Linear Non-Gaussian Acyclic Model)**: This method extends independent component analysis (ICA) to causal discovery for linear, acyclic models where the noise terms are non-Gaussian. Unlike PC, LiNGAM can identify the full causal order (up to scaling and permutation) without relying on conditional independence tests or an independence oracle. While it assumes linearity and acyclicity, its ability to leverage non-Gaussianity can be powerful in scenarios where PC might struggle due to high-dimensional conditioning sets.

5.  **Time-Series Extensions of PC**: While standard PC is not ideal, researchers have developed extensions that incorporate temporal information, such as `PCMCI` (PC algorithm for Moment-based Conditional Independence) or `Granger-PC`. These methods adapt the constraint-based framework to explicitly handle temporal dependencies and lagged effects, making them more appropriate for time-series data while retaining some of the advantages of the PC algorithm.

In summary, while `StandardScaler` and duplicate removal are generally benign preprocessing steps, the fundamental nature of EEG data (noise, non-stationarity, temporal dynamics) inherently limits the efficacy of standard PC. For a more robust and biologically meaningful causal inference in EEG, methods explicitly designed for time-series analysis and capable of handling non-stationarity and non-linearity are often more appropriate.

## Outline Custom Preprocessing Strategy (if needed)

### Subtask:
If custom preprocessing is deemed necessary, outline a strategy for downloading and preparing the raw CHBMIT dataset. This step will cover key preprocessing steps relevant for causal inference, such as artifact removal, bandpass filtering, and segmentation, ensuring that the data retains information important for discovering causal relationships.


## Outline Custom Preprocessing Strategy for Raw CHBMIT Dataset

Given that the current preprocessed `chbmit_preprocessed_data.csv` might have lost critical temporal and spectral information essential for comprehensive causal inference, a custom preprocessing strategy for the raw CHBMIT dataset is necessary. This strategy aims to preserve and highlight these dynamics, making the data suitable for more advanced time-series causal discovery methods.

### 1. Data Acquisition: Downloading the Raw CHBMIT Dataset

The CHBMIT Scalp EEG Database is publicly available on [PhysioNet](https://physionet.org/content/chbmit/1.0.0/). The dataset consists of continuous EEG recordings from pediatric subjects with intractable seizures. It is provided in EDF (European Data Format) files, which contain raw EEG signals along with metadata.

**Steps:**
1.  **Access PhysioNet:** Navigate to the CHBMIT Scalp EEG Database on PhysioNet.
2.  **Download EDF Files:** Download the raw EDF files for the desired subjects. This often requires registration and agreement to the usage terms.
3.  **Metadata:** Ensure to also download and parse associated metadata (e.g., `.txt` files) which contain information about seizure onset times, channel labels, and sampling frequencies.

### 2. Key Preprocessing Steps for Causal Inference

This section details preprocessing steps that focus on cleaning the raw EEG data while retaining the nuanced temporal and spectral information crucial for causal discovery.

#### a. Initial Data Loading and Inspection

*   **Load EDFs:** Use libraries like `mne-python` to load the `.edf` files. MNE provides robust tools for handling EEG/MEG data.
*   **Channel Selection:** Select relevant EEG channels and drop any external channels (e.g., EKG, EOG) not intended for analysis, or use them for artifact detection.
*   **Sampling Rate:** Note the original sampling frequency of the recordings, as this dictates the temporal resolution.

#### b. Artifact Removal

Artifacts (e.g., eye blinks, muscle activity, line noise) can introduce spurious correlations and obscure true neural dependencies. Effective artifact removal is paramount.

*   **Line Noise Filtering:** Apply a notch filter (e.g., at 50 Hz or 60 Hz, depending on geographical location) to remove power line interference.
*   **Ocular Artifacts (EOG):** Implement Independent Component Analysis (ICA) or regression-based methods to identify and remove components related to eye movements and blinks. MNE's ICA implementation is highly effective.
*   **Muscle Artifacts (EMG):** While harder to remove completely without affecting neural signals, methods like ICA can help. Alternatively, segments with high EMG activity can be identified and excluded, or specific frequency ranges associated with muscle activity can be attenuated.
*   **Bad Channels/Epochs:** Visually inspect data or use automated methods (e.g., based on amplitude thresholds, variance) to identify and interpolate or reject noisy channels/epochs.

#### c. Bandpass Filtering

Bandpass filtering helps isolate specific frequency bands and remove very slow drifts and very high-frequency noise, which are often non-physiological or not relevant for the causal questions.

*   **Typical Range:** Apply a bandpass filter (e.g., 0.5 - 45 Hz or 1 - 70 Hz) to focus on the physiologically relevant EEG spectrum. This removes slow drifts (below 0.5 Hz) and high-frequency noise (above 45-70 Hz).
*   **Preservation of Neural Frequencies:** Ensure that the filter cut-offs are chosen to preserve the major neural oscillations (delta, theta, alpha, beta, gamma) which are often implicated in brain dynamics and causal interactions. This is crucial as causal effects can be frequency-specific.

#### d. Re-referencing

EEG signals are recorded with respect to a reference electrode. Re-referencing can significantly impact the spatial distribution of signals and, consequently, observed causal links.

*   **Common Average Reference (CAR):** Subtract the average activity across all electrodes from each electrode. This is a common method that can reduce noise common to all channels.
*   **Reference Electrode Standardization Technique (REST):** A more sophisticated method that estimates a reference at infinity, which can provide a more accurate representation of brain activity.
*   **Bipolar/Laplacian:** For specific analyses, creating bipolar montages or applying Laplacian (source) derivations can enhance spatial specificity and reduce volume conduction effects, which are critical for inferring direct causal influences between distinct brain regions.

#### e. Segmentation (Epoching)

Segmenting continuous EEG data into meaningful epochs is essential for analyzing specific states or events.

*   **Event-Related Segmentation:** Epoch the data around events of interest, such as seizure onset markers (from metadata) or specific cognitive tasks.
*   **State-Based Segmentation:** For inter-ictal and pre-ictal analysis, segment continuous data into fixed-duration, non-overlapping windows. Crucially, *maintain the temporal order of segments within each state*. For instance, a pre-ictal state could be defined as a continuous block of X minutes leading up to a seizure. This preserves the temporal sequence within the state, which is vital for time-series causal inference.
*   **Temporal Continuity:** Unlike the current approach which treats each row as an i.i.d. sample, ensure that when segments are created, their temporal relationship (e.g., segment N precedes segment N+1) is maintained or at least considered. This allows for the investigation of time-lagged causal effects.

#### f. Resampling (Downsampling)

Raw EEG data often has high sampling rates (e.g., 256 Hz, 512 Hz). Downsampling can reduce computational load but must be done carefully.

*   **Purpose:** Reduce the sampling rate to a more manageable level (e.g., 100-200 Hz) while preserving relevant frequency content. Downsampling too aggressively can lead to aliasing and loss of high-frequency information.
*   **Anti-aliasing Filter:** Always apply an anti-aliasing low-pass filter before downsampling to prevent spectral folding.
*   **Impact on Temporal Resolution:** Acknowledge the trade-off: downsampling reduces the temporal resolution, which might affect the detection of very fast, short-lagged causal interactions. The chosen sampling rate should still be high enough to capture the fastest relevant neural dynamics.

### 3. Retaining Crucial Information for Causal Discovery

This custom preprocessing strategy is designed to mitigate the potential loss of temporal and spectral information identified in the existing preprocessed data:

*   **Temporal Information:** By explicitly segmenting continuous data with an awareness of temporal order and avoiding treating each time point as an i.i.d. snapshot, we retain the sequential nature of EEG. This allows for the detection of time-lagged causal effects (e.g., Granger causality).
*   **Spectral Information:** Careful bandpass filtering and avoiding aggregation over frequency bands ensure that the power and phase relationships within different oscillations are preserved. This enables causal discovery methods that can operate on frequency-specific measures or investigate cross-frequency coupling.
*   **Reduced Volume Conduction:** Re-referencing techniques like REST or Laplacian derivations help minimize the effects of volume conduction, leading to more accurate estimates of direct neural interactions rather than spurious correlations due to shared signal spread.

### 4. Preparing for Advanced Time-Series Causal Inference

By implementing this strategy, the preprocessed data becomes suitable for a range of advanced causal inference methods that explicitly model temporal dynamics:

*   **Granger Causality:** Can be applied to the segmented, clean time series data to infer directional causal influences over time.
*   **Dynamic Causal Modeling (DCM):** A more model-driven approach that requires biologically plausible models of neural interactions, typically applied to source-localized data, which can be derived from the preprocessed EEG.
*   **Time-Series Extensions of PC:** Algorithms like PC-LST (PC-Lagged with Sparse Time-series) or other variants that incorporate time lags into conditional independence tests can be applied to infer dynamic causal graphs.
*   **Multivariate Autoregressive (MVAR) Models:** These models can capture the dynamic interactions between multiple brain regions over time, providing insights into causal flow.

This comprehensive preprocessing framework, though more intensive, ensures that the resulting data maximizes the potential for uncovering genuine, nuanced causal relationships within the CHBMIT EEG dataset, addressing the limitations of applying static causal inference methods to overly aggregated data.

## Outline Custom Preprocessing Strategy for Raw CHBMIT Dataset

Given that the current preprocessed `chbmit_preprocessed_data.csv` might have lost critical temporal and spectral information essential for comprehensive causal inference, a custom preprocessing strategy for the raw CHBMIT dataset is necessary. This strategy aims to preserve and highlight these dynamics, making the data suitable for more advanced time-series causal discovery methods.

### 1. Data Acquisition: Downloading the Raw CHBMIT Dataset

The CHBMIT Scalp EEG Database is publicly available on [PhysioNet](https://physionet.org/content/chbmit/1.0.0/). The dataset consists of continuous EEG recordings from pediatric subjects with intractable seizures. It is provided in EDF (European Data Format) files, which contain raw EEG signals along with metadata.

**Steps:**
1.  **Access PhysioNet:** Navigate to the CHBMIT Scalp EEG Database on PhysioNet.
2.  **Download EDF Files:** Download the raw EDF files for the desired subjects. This often requires registration and agreement to the usage terms.
3.  **Metadata:** Ensure to also download and parse associated metadata (e.g., `.txt` files) which contain information about seizure onset times, channel labels, and sampling frequencies.

### 2. Key Preprocessing Steps for Causal Inference

This section details preprocessing steps that focus on cleaning the raw EEG data while retaining the nuanced temporal and spectral information crucial for causal discovery.

#### a. Initial Data Loading and Inspection

*   **Load EDFs:** Use libraries like `mne-python` to load the `.edf` files. MNE provides robust tools for handling EEG/MEG data.
*   **Channel Selection:** Select relevant EEG channels and drop any external channels (e.g., EKG, EOG) not intended for analysis, or use them for artifact detection.
*   **Sampling Rate:** Note the original sampling frequency of the recordings, as this dictates the temporal resolution.

#### b. Artifact Removal

Artifacts (e.g., eye blinks, muscle activity, line noise) can introduce spurious correlations and obscure true neural dependencies. Effective artifact removal is paramount.

*   **Line Noise Filtering:** Apply a notch filter (e.g., at 50 Hz or 60 Hz, depending on geographical location) to remove power line interference.
*   **Ocular Artifacts (EOG):** Implement Independent Component Analysis (ICA) or regression-based methods to identify and remove components related to eye movements and blinks. MNE's ICA implementation is highly effective.
*   **Muscle Artifacts (EMG):** While harder to remove completely without affecting neural signals, methods like ICA can help. Alternatively, segments with high EMG activity can be identified and excluded, or specific frequency ranges associated with muscle activity can be attenuated.
*   **Bad Channels/Epochs:** Visually inspect data or use automated methods (e.g., based on amplitude thresholds, variance) to identify and interpolate or reject noisy channels/epochs.

#### c. Bandpass Filtering

Bandpass filtering helps isolate specific frequency bands and remove very slow drifts and very high-frequency noise, which are often non-physiological or not relevant for the causal questions.

*   **Typical Range:** Apply a bandpass filter (e.g., 0.5 - 45 Hz or 1 - 70 Hz) to focus on the physiologically relevant EEG spectrum. This removes slow drifts (below 0.5 Hz) and high-frequency noise (above 45-70 Hz).
*   **Preservation of Neural Frequencies:** Ensure that the filter cut-offs are chosen to preserve the major neural oscillations (delta, theta, alpha, beta, gamma) which are often implicated in brain dynamics and causal interactions. This is crucial as causal effects can be frequency-specific.

#### d. Re-referencing

EEG signals are recorded with respect to a reference electrode. Re-referencing can significantly impact the spatial distribution of signals and, consequently, observed causal links.

*   **Common Average Reference (CAR):** Subtract the average activity across all electrodes from each electrode. This is a common method that can reduce noise common to all channels.
*   **Reference Electrode Standardization Technique (REST):** A more sophisticated method that estimates a reference at infinity, which can provide a more accurate representation of brain activity.
*   **Bipolar/Laplacian:** For specific analyses, creating bipolar montages or applying Laplacian (source) derivations can enhance spatial specificity and reduce volume conduction effects, which are critical for inferring direct causal influences between distinct brain regions.

#### e. Segmentation (Epoching)

Segmenting continuous EEG data into meaningful epochs is essential for analyzing specific states or events.

*   **Event-Related Segmentation:** Epoch the data around events of interest, such as seizure onset markers (from metadata) or specific cognitive tasks.
*   **State-Based Segmentation:** For inter-ictal and pre-ictal analysis, segment continuous data into fixed-duration, non-overlapping windows. Crucially, *maintain the temporal order of segments within each state*. For instance, a pre-ictal state could be defined as a continuous block of X minutes leading up to a seizure. This preserves the temporal sequence within the state, which is vital for time-series causal inference.
*   **Temporal Continuity:** Unlike the current approach which treats each row as an i.i.d. sample, ensure that when segments are created, their temporal relationship (e.g., segment N precedes segment N+1) is maintained or at least considered. This allows for the investigation of time-lagged causal effects.

#### f. Resampling (Downsampling)

Raw EEG data often has high sampling rates (e.g., 256 Hz, 512 Hz). Downsampling can reduce computational load but must be done carefully.

*   **Purpose:** Reduce the sampling rate to a more manageable level (e.g., 100-200 Hz) while preserving relevant frequency content. Downsampling too aggressively can lead to aliasing and loss of high-frequency information.
*   **Anti-aliasing Filter:** Always apply an anti-aliasing low-pass filter before downsampling to prevent spectral folding.
*   **Impact on Temporal Resolution:** Acknowledge the trade-off: downsampling reduces the temporal resolution, which might affect the detection of very fast, short-lagged causal interactions. The chosen sampling rate should still be high enough to capture the fastest relevant neural dynamics.

### 3. Retaining Crucial Information for Causal Discovery

This custom preprocessing strategy is designed to mitigate the potential loss of temporal and spectral information identified in the existing preprocessed data:

*   **Temporal Information:** By explicitly segmenting continuous data with an awareness of temporal order and avoiding treating each time point as an i.i.d. snapshot, we retain the sequential nature of EEG. This allows for the detection of time-lagged causal effects (e.g., Granger causality).
*   **Spectral Information:** Careful bandpass filtering and avoiding aggregation over frequency bands ensure that the power and phase relationships within different oscillations are preserved. This enables causal discovery methods that can operate on frequency-specific measures or investigate cross-frequency coupling.
*   **Reduced Volume Conduction:** Re-referencing techniques like REST or Laplacian derivations help minimize the effects of volume conduction, leading to more accurate estimates of direct neural interactions rather than spurious correlations due to shared signal spread.

### 4. Preparing for Advanced Time-Series Causal Inference

By implementing this strategy, the preprocessed data becomes suitable for a range of advanced causal inference methods that explicitly model temporal dynamics:

*   **Granger Causality:** Can be applied to the segmented, clean time series data to infer directional causal influences over time.
*   **Dynamic Causal Modeling (DCM):** A more model-driven approach that requires biologically plausible models of neural interactions, typically applied to source-localized data, which can be derived from the preprocessed EEG.
*   **Time-Series Extensions of PC:** Algorithms like PC-LST (PC-Lagged with Sparse Time-series) or other variants that incorporate time lags into conditional independence tests can be applied to infer dynamic causal graphs.
*   **Multivariate Autoregressive (MVAR) Models:** These models can capture the dynamic interactions between multiple brain regions over time, providing insights into causal flow.

This comprehensive preprocessing framework, though more intensive, ensures that the resulting data maximizes the potential for uncovering genuine, nuanced causal relationships within the CHBMIT EEG dataset, addressing the limitations of applying static causal inference methods to overly aggregated data.

## Address Space and Memory Constraints

### Subtask:
Provide practical strategies for preprocessing a large dataset like CHBMIT without running out of space or memory. This includes techniques like processing data in chunks, selective downsampling, using memory-efficient data structures, and storing intermediate results efficiently. We will also discuss the implications of these choices on causal inference.


## Address Space and Memory Constraints

### Practical Strategies for Handling Large EEG Datasets

Working with large EEG datasets like CHBMIT often presents significant memory and space challenges during preprocessing. Efficient strategies are crucial to avoid running out of RAM or disk space while preparing the data for causal inference. Below are several practical techniques and their implications.

#### 1. Processing Data in Manageable Chunks

Instead of loading the entire dataset into memory, processing it in smaller, manageable chunks is a fundamental strategy. This can be implemented in several ways:

*   **Subject-by-Subject**: If the dataset consists of recordings from multiple subjects, process one subject's data at a time. Load all files for a single subject, preprocess them, extract features, and save the results before moving to the next subject.
*   **File-by-File**: For datasets where each recording session is stored in a separate file (e.g., individual EEG recordings), process each file sequentially. This is often applicable when a single subject has multiple recording sessions.
*   **Time-Window-by-Time-Window**: Within a single long recording, data can be processed in fixed-duration time windows (e.g., 5-second segments). Features are extracted from each window, and the windows are processed iteratively.

**Implications for Causal Inference**: Chunking is generally benign if the causal inference is performed *within* each chunk and then aggregated or if the chunks are representative samples. However, if causal links span across chunk boundaries (e.g., a causal effect originating in one time window and manifesting in the next), simple chunking might break these dependencies. When aggregating results, care must be taken to ensure the statistical methods are appropriate for combined inferences from multiple independent analyses.

#### 2. Selective Downsampling

EEG data is often acquired at high sampling rates (e.g., 250 Hz, 500 Hz, or more). Downsampling can drastically reduce memory and storage requirements.

*   **Method**: Reduce the sampling frequency of the EEG signals. For example, downsampling from 500 Hz to 100 Hz reduces the data points by a factor of 5.
*   **Trade-offs with Temporal Resolution**: The main trade-off is the loss of temporal resolution. Downsampling limits the highest frequency components that can be analyzed (Nyquist frequency). If very fast neurological events or short-lagged causal interactions (e.g., within milliseconds) are crucial, aggressive downsampling can obscure these. However, for slower cortical dynamics or analyses focusing on broader frequency bands, downsampling can be acceptable or even beneficial by removing high-frequency noise.

**Implications for Causal Inference**: Downsampling directly impacts the ability to detect rapid causal effects. Causal links with short time lags might become undetectable or appear as instantaneous effects if the time resolution is insufficient. It's crucial to consider the expected temporal scale of the causal phenomena being investigated when deciding on a downsampling factor.

#### 3. Using Memory-Efficient Data Structures

Choosing the right data structures can significantly reduce memory footprint.

*   **NumPy Arrays**: Python lists are memory-inefficient for numerical data. Using NumPy arrays is essential as they store data in a contiguous block of memory, offering lower memory usage and faster operations.
*   **`h5py` for Out-of-Core Processing**: For datasets that are too large to fit in memory even with NumPy, `h5py` allows interaction with HDF5 files on disk as if they were NumPy arrays. This enables out-of-core computations where only a small portion of data is loaded into RAM at any given time.
*   **Sparse Matrices**: If the extracted features are sparse (contain many zero values), using sparse matrix formats (e.g., from `scipy.sparse`) can save considerable memory.

**Implications for Causal Inference**: These are primarily technical optimizations that enable processing of larger datasets. They do not directly alter the causal inference logic but ensure that the PC algorithm (or other methods) can actually run on the data without memory crashes. Efficient data structures contribute to faster processing, which can be critical for iterative causal discovery workflows.

#### 4. Efficient Storage of Intermediate Results

After each major preprocessing step (e.g., artifact removal, feature extraction, state separation), saving the intermediate results to disk in an efficient format is good practice.

*   **Formats**: Consider formats like:
    *   `.fif` (MEG/EEG data format, often used with MNE-Python for raw/processed signals).
    *   `.hdf5` (Hierarchical Data Format 5): Excellent for storing large numerical datasets with metadata, supporting compression and fast access to subsets of data.
    *   `.parquet`: Columnar storage format, highly efficient for tabular data, especially with large numbers of rows, and supports schema evolution and compression.
    *   `.npy` or `.npz`: For saving raw NumPy arrays efficiently.
*   **Benefits**: Prevents data loss, allows resuming processing from an intermediate step, reduces re-computation time, and facilitates sharing of processed data.

**Implications for Causal Inference**: Storing intermediate results enhances reproducibility and makes the causal inference pipeline more robust. If a causal discovery step fails or needs parameter tuning, having preprocessed stages saved means one doesn't have to restart from raw data. It also allows for easier experimentation with different causal inference algorithms or parameters on the same preprocessed data.

By carefully applying these strategies, researchers can effectively manage the memory and space requirements of large EEG datasets, enabling complex analyses like causal inference that would otherwise be computationally prohibitive.

## Final Task

### Subtask:
Summarize the findings regarding the current dataset and provide a conclusive recommendation on whether to proceed with custom preprocessing and how to manage the process effectively.


## Summary:

### Q&A
**Should we proceed with custom preprocessing and how should we manage the process effectively?**
Yes, custom preprocessing is highly recommended and necessary. The current preprocessing, while standard, likely stripped critical temporal and spectral information from the CHBMIT EEG dataset, leading to an empty causal graph when applying the PC algorithm. To manage the process effectively, a comprehensive strategy involving careful raw data acquisition, detailed artifact removal, bandpass filtering, re-referencing, segmentation that preserves temporal continuity, and selective downsampling will be crucial. Memory and space constraints can be managed through data chunking, memory-efficient data structures, and efficient storage of intermediate results.

### Data Analysis Key Findings
*   **Empty Causal Graph**: The PC algorithm, when applied to the inter-ictal state data using the existing preprocessing and alpha level, resulted in an **empty directed causal graph and an empty skeleton**. This indicates that the algorithm found **no statistically significant causal links** under the given conditions, rather than an issue with graph conversion.
*   **Impact of Existing Preprocessing**:
    *   **StandardScaler** and **duplicate column removal** were deemed benign, preserving linear relationships and simplifying the data without information loss for conditional independence tests using `hsic`.
    *   However, the initial creation of the `chbmit_preprocessed_data.csv` likely led to a **significant loss of temporal and spectral resolution**, which is critical for EEG causal inference. The data appears to be treated as i.i.d. samples, violating the inherent temporal dependencies of EEG.
*   **Inherent Challenges of EEG Data for PC Algorithm**: EEG datasets present fundamental challenges for constraint-based algorithms like the PC algorithm due to:
    *   **High Noise Levels**: Obscuring genuine causal relationships.
    *   **Non-stationarity**: Causal relationships in EEG change over time, violating PC's stationarity assumption.
    *   **Complex Multivariate Relationships**: Non-linear, time-lagged, and feedback loop-driven interactions are not fully captured by standard PC.
    *   **Temporal Dependencies**: Standard PC primarily infers contemporaneous relationships, missing crucial time-lagged effects.
*   **Trade-offs of Preprocessed vs. Raw Data**: While preprocessed data offers benefits like reduced noise, dimensionality reduction, and computational efficiency, it comes at the cost of losing vital temporal and spectral resolution, obscuring non-linearities, and violating i.i.d. assumptions for time-series data. Raw data preserves full information but is high-dimensional and complex.
*   **Need for Custom Preprocessing**: To address the limitations, a custom preprocessing strategy for raw CHBMIT data is essential. This includes artifact removal (notch filtering, ICA for ocular artifacts), bandpass filtering for relevant frequencies, re-referencing (CAR, REST, Laplacian) to reduce volume conduction, segmentation that *maintains temporal order*, and careful resampling.
*   **Memory and Space Management**: For large datasets, practical strategies include processing data in manageable chunks (subject-by-subject, file-by-file, time-window-by-time-window), selective downsampling with anti-aliasing, using memory-efficient data structures (NumPy, `h5py`, sparse matrices), and efficiently storing intermediate results (`.fif`, `.hdf5`, `.parquet`).

### Insights or Next Steps
*   **Implement Custom Preprocessing**: Proceed with the outlined custom preprocessing strategy for the raw CHBMIT dataset, focusing on preserving temporal and spectral information and ensuring appropriate handling of time series characteristics. This is a critical first step to enable meaningful causal inference in EEG.
*   **Explore Alternative Causal Inference Methods**: Given the limitations of the standard PC algorithm for EEG data, investigate and apply time-series-specific causal inference methods such as Granger Causality, Transfer Entropy, Dynamic Causal Modeling (DCM), or time-series extensions of PC (e.g., `PCMCI`) once the data is appropriately preprocessed.


# Task
Install `mne` library for EEG data loading and preprocessing.

## Install MNE-Python

### Subtask:
Install the `mne` library, which is essential for loading and preprocessing raw EEG data (EDF files). This library provides robust tools for neurophysiological signal processing. This step does not significantly impact memory or space.


**Reasoning**:
Install the mne library using pip, which is essential for loading and preprocessing raw EEG data.



In [7]:
pip install mne

## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.


## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.

## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.

## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.


## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.


## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.



## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.



## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.


## Acquire Raw CHBMIT Data (Chunk-wise)

### Subtask:
Manually download portions of the raw CHBMIT dataset from PhysioNet. To manage space, download data in manageable chunks (e.g., subject-by-subject or a few files at a time) rather than the entire dataset simultaneously. Store these raw files in a designated local directory.

#### Instructions
1. Access the CHBMIT Scalp EEG Database on PhysioNet (https://physionet.org/content/chbmit/1.0.0/).
2. Download the raw EDF files for a few selected subjects or a manageable subset of the data. You may need to register and agree to their usage terms.
3. Create a local directory (e.g., `/content/data/raw/chbmit/`) to store these downloaded raw EDF files.
4. Store the downloaded EDF files, along with any associated metadata (e.g., `.txt` files containing seizure onset times), into this newly created directory.


In [8]:
import mne
import os

print("MNE-Python and os libraries imported successfully.")

MNE-Python and os libraries imported successfully.


In [29]:
import mne
import os

mne.set_log_level('WARNING') # Set log level to WARNING to reduce verbose output

raw_data_dir = '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.01_01.edf'
os.makedirs(raw_data_dir, exist_ok=True)

# --- USER ACTION REQUIRED ---
# Please manually download EDF files from PhysioNet and place them in the directory specified above.
# Example: /content/data/raw/chbmit/chb01_01.edf
# --------------------------

# List all EDF files in the directory
edf_files = [os.path.join(raw_data_dir, f) for f in os.listdir(raw_data_dir) if f.endswith('.edf')]

if not edf_files:
    print(f"No EDF files found in {raw_data_dir}. Please download and place the raw CHBMIT EDF files there.")
else:
    print(f"Found {len(edf_files)} EDF files in {raw_data_dir}.")
    print(f"Total {len(edf_files)} EDF files loaded for processing.")
    print("Example files:", edf_files[:3]) # Print first 3 files if many

# List to store preprocessed MNE Raw objects (or extracted data)
preprocessed_raw_objects = []

# Define preprocessing parameters
low_freq = 0.5
high_freq = 45
new_sfreq = 100 # Hz

if edf_files:
    for i, edf_file in enumerate(edf_files):
        print(f"\nProcessing file {i+1}/{len(edf_files)}: {os.path.basename(edf_file)}")
        try:
            # 5a. Load the EDF file into an MNE Raw object
            raw = mne.io.read_raw_edf(edf_file, preload=True)
            # Ensure a descriptive name is set for saving later
            raw.info['description'] = os.path.basename(edf_file).replace('.edf', '')
            print(f"  Original sampling frequency: {raw.info['sfreq']} Hz")
            print(f"  Number of channels: {len(raw.ch_names)}")
            print(f"  Duration: {raw.times[-1] / 60:.2f} minutes")

            # Apply common average reference to EEG channels only if they exist
            eeg_channels = mne.pick_types(raw.info, eeg=True, exclude='bads')
            if len(eeg_channels) > 0:
                raw.set_eeg_reference(ref_channels='average')
                print("  Applied Common Average Reference.")
            else:
                print("  No EEG channels found for re-referencing.")

            # 5b. Apply a bandpass filter
            raw.filter(l_freq=low_freq, h_freq=high_freq, picks='eeg')
            print(f"  Applied bandpass filter ({low_freq}-{high_freq} Hz).")

            # 5c. Downsample the Raw object
            raw.resample(sfreq=new_sfreq)
            print(f"  Downsampled to {raw.info['sfreq']} Hz.")

            # 5d. Store the preprocessed Raw object (or its data)
            preprocessed_raw_objects.append(raw) # Store the MNE Raw object

        except Exception as e:
            print(f"  Error processing {os.path.basename(edf_file)}: {e}")

    print(f"\nFinished preprocessing {len(preprocessed_raw_objects)} files.")
else:
    print("Skipping preprocessing as no EDF files were found.")

Found 6 EDF files in /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.01_01.edf.
Total 6 EDF files loaded for processing.
Example files: ['/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.01_01.edf/chb01_12.edf', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.01_01.edf/chb01_11.edf', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.01_01.edf/chb01_13.edf']

Processing file 1/6: chb01_12.edf


/tmp/ipython-input-2891246402.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 2/6: chb01_11.edf


/tmp/ipython-input-2891246402.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 3/6: chb01_13.edf


...

  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 5/6: chb01_15.edf


/tmp/ipython-input-2891246402.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 6/6: chb01_16.edf


/tmp/ipython-input-2891246402.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Finished preprocessing 6 files.


In [13]:
import mne
print(f"MNE-Python version: {mne.__version__}")

MNE-Python version: 1.11.0


In [21]:
print(ica_cleaned_fif_paths)
print(f"Total ICA-cleaned .fif files saved: {len(ica_cleaned_fif_paths)}")

['/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/unnamed_raw_0_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/unnamed_raw_1_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/unnamed_raw_2_ica_cleaned.fif']
Total ICA-cleaned .fif files saved: 3


In [30]:
from mne.preprocessing import ICA


processed_data_dir = '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed'
os.makedirs(processed_data_dir, exist_ok=True)


ica_cleaned_fif_paths = []


if preprocessed_raw_objects:
    print(f"\nApplying ICA to {len(preprocessed_raw_objects)} preprocessed Raw objects and saving them...")
    for i, raw_obj in enumerate(preprocessed_raw_objects):
        # Use the description set in the previous step, or a generic name if unavailable
        original_filename = raw_obj.info['description'] if raw_obj.info['description'] else f'unnamed_raw_{i}'
        output_fif_path = os.path.join(processed_data_dir, f'{original_filename}_ica_cleaned.fif')

        # Check if ICA-cleaned file already exists to avoid reprocessing
        if os.path.exists(output_fif_path):
            print(f"  Skipping ICA for {original_filename} as {output_fif_path} already exists.")

            continue

        print(f"  Processing raw object {i+1}/{len(preprocessed_raw_objects)}: {original_filename}")
        try:

            n_components_ica = min(20, len(raw_obj.ch_names))
            ica = ICA(n_components=n_components_ica, random_state=99, verbose=False)


            ica.fit(raw_obj)


            ica_cleaned_raw = ica.apply(raw_obj.copy(), exclude=ica.exclude, verbose=False)
            print(f"    ICA applied. Original channels: {len(raw_obj.ch_names)}, ICA components: {ica.n_components_}, Excluded components: {len(ica.exclude)}")


            ica_cleaned_raw.set_meas_date(None)



            ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)
            print(f"    Saved ICA-cleaned data to: {output_fif_path}")

        except Exception as e:
            print(f"  Error applying ICA to raw object {i+1} ({original_filename}): {e}")


ica_cleaned_fif_paths = [os.path.join(processed_data_dir, f) for f in os.listdir(processed_data_dir) if f.endswith('_ica_cleaned.fif') or f.endswith('_ica_cleaned.fif.gz')]

if ica_cleaned_fif_paths:
    print(f"\nFinished ICA cleaning and saving. Total {len(ica_cleaned_fif_paths)} ICA-cleaned .fif files found on disk.")
    print("List of ICA-cleaned .fif files:", ica_cleaned_fif_paths)
else:
    print("\nNo ICA-cleaned .fif files were found on disk. Please ensure EDF files are present in the raw data directory and processed in previous steps, and ICA runs successfully.")


Applying ICA to 6 preprocessed Raw objects and saving them...
  Processing raw object 1/6: chb01_12
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-2294146930.py:43: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_12_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_12_ica_cleaned.fif
  Processing raw object 2/6: chb01_11
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-2294146930.py:43: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_11_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_11_ica_cleaned.fif
  Processing raw object 3/6: chb01_13
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


...

    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_14_ica_cleaned.fif
  Processing raw object 5/6: chb01_15
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-2294146930.py:43: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_15_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_15_ica_cleaned.fif
  Processing raw object 6/6: chb01_16
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-2294146930.py:43: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_16_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_16_ica_cleaned.fif

Finished ICA cleaning and saving. Total 16 ICA-cleaned .fif files found on disk.
List of ICA-cleaned .fif files: ['/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_08_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_01_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_02_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_09_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_03_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_10_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_04_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_05_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_C

In [ ]:
import numpy as np

# 1. Define segmentation parameters
segment_duration = 5 # seconds

# 2. Initialize an empty list to store the extracted data from each segment
segmented_data_for_causal_analysis = []

# 3. Iterate through each ICA-cleaned .fif file path
if 'ica_cleaned_fif_paths' in locals() and ica_cleaned_fif_paths:
    print(f"\nSegmenting data from {len(ica_cleaned_fif_paths)} ICA-cleaned .fif files...")
    for i, fif_path in enumerate(ica_cleaned_fif_paths):
        filename = os.path.basename(fif_path)
        print(f"  Segmenting file {i+1}/{len(ica_cleaned_fif_paths)}: {filename}")
        try:
            # Load the ICA-cleaned Raw object from the .fif file
            raw_obj = mne.io.read_raw_fif(fif_path, preload=True, verbose=False)

            # 4. Create non-overlapping epochs (segments) of the defined duration
            epochs = mne.make_fixed_length_epochs(raw_obj, duration=segment_duration, overlap=0, preload=True, verbose=False)
            print(f"    Created {len(epochs)} epochs, each {segment_duration} seconds long.")

            # 5. Extract the numerical data from each epoch and prepare for causal inference
            epochs_data = epochs.get_data(picks='eeg')

            n_epochs, n_channels, n_times = epochs_data.shape
            reshaped_epochs_data = epochs_data.reshape(n_epochs, n_channels * n_times)

            segmented_data_for_causal_analysis.append(reshaped_epochs_data)

            # Explicitly delete objects to free memory after processing a chunk
            del epochs, raw_obj

        except Exception as e:
            print(f"  Error segmenting file {i+1} ({filename}): {e}")

    # Concatenate all segmented data if multiple files were processed
    if segmented_data_for_causal_analysis:
        all_segmented_data = np.vstack(segmented_data_for_causal_analysis)
        print(f"\nFinished segmentation. Processed {len(ica_cleaned_fif_paths)} files and created a total of {all_segmented_data.shape[0]} segments.")
        print(f"Shape of combined segmented data for causal analysis: {all_segmented_data.shape}")
    else:
        print("No segments were successfully created for causal analysis.")
else:
    print("\nNo ICA-cleaned .fif file paths available for segmentation. Please ensure ICA cleaning and saving steps ran successfully.")

In [ ]:
import h5py
import os

# 1. Check if segmented data exists
if 'all_segmented_data' in locals() and all_segmented_data.size > 0:
    data_to_save = all_segmented_data
    print(f"Data to save found with shape: {data_to_save.shape}")
elif segmented_data_for_causal_analysis:

    data_to_save = np.vstack(segmented_data_for_causal_analysis)
    print(f"Data to save found in list form with shape: {data_to_save.shape}")
else:
    print("No segmented data available to save. Please ensure previous steps ran successfully.")
    data_to_save = None

if data_to_save is not None:
    # 2. Define filename and path
    processed_data_dir = '/content/data/processed/'
    os.makedirs(processed_data_dir, exist_ok=True)
    output_filename = os.path.join(processed_data_dir, 'segmented_eeg_data.h5')

    # 4. Use h5py.File() to create or open an HDF5 file
    with h5py.File(output_filename, 'w') as f:
        # 5. Save the NumPy array as a dataset within the HDF5 file with compression
        f.create_dataset('eeg_segments', data=data_to_save, compression="gzip", compression_opts=9)

    print(f"Successfully saved segmented EEG data to: {output_filename}")
    print(f"Saved data shape: {data_to_save.shape}")
